In [1]:
import numpy as np
import pandas as pd
from sunpy.physics.differential_rotation import rot_hpc
from astropy import units as u
from shapely.geometry import Polygon, Point
from shapely import wkt
import os
import datetime
from sunpy.time import parse_time

In [20]:
def associate_sigmoid(inputFile_fl, inputFile_sigmoid, spatial_sep, output2file=False, out_file = None):
    #import a record of flare events as a DataFrame
    flare_set = pd.read_csv(inputFile_fl, delimiter = ',', header = 0)
    #import a record of sigmoid events as a DataFrame
    sigmoid_set = pd.read_csv(inputFile_sigmoid, delimiter = ',', header = 0)
    #set solar radius
    r = 966
    #how many flare events working with
    length = flare_set.shape[0]
    #list of zeroes with length of number of flare events
    zeroes = [0 for i in range(length)]
    #list of nulls with length of number of flare events
    nones = [None for i in range(length)]
    #create columns (filled with zeroes) for tracking associated events
    flare_set.loc[:, 'associated_sigmoid'] = nones
    flare_set.loc[:, 'num_associated_sigmoid'] = zeroes
    flare_set.loc[:, 'dist_sigmoid'] = zeroes

    sigmoid_set = sigmoid_set.rename(columns={'event_starttime':'starttime_sigmoid', 'event_endtime':'endtime_sigmoid',
                                              'hpc_bbox':'hpc_bbox_sigmoid', 'ar_noaanum': 'sigmoid_noaanum',
                                             'search_channelid': 'search_channelid_sigmoid'})
    sigmoid_keywords = ['starttime_sigmoid', 'endtime_sigmoid','hpc_bbox_sigmoid', 'sigmoid_noaanum', 
                        'search_channelid_sigmoid']
   
    #create columns filled with Nones for each sigmoid keyword
    for elem in sigmoid_keywords:
         flare_set.loc[:, elem] = nones
    #convert start and end times to datetime objects
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
    sigmoid_set['starttime_sigmoid'] = map(parse_time, sigmoid_set['starttime_sigmoid'])
    sigmoid_set['endtime_sigmoid'] = map(parse_time, sigmoid_set['endtime_sigmoid'])

    #set positional row index 
    i = -1

    for annoying_wrong_obj_type in flare_set['event_starttime']:
        i += 1
        #print which flare event function is currently processing, so the user has an idea of how much longer
        #program will need to run
        start = flare_set['event_starttime'].values[i]
        end = flare_set['event_endtime'].values[i]
        if (i+1)%25 == 0:
            print '%d / %d events' %((i+1), length)
        #begin eliminating sigmoid events based on temporal parameters
        sigmoid_search = sigmoid_set.ix[sigmoid_set['starttime_sigmoid']<=start]
        sigmoid_search = sigmoid_search.ix[sigmoid_search['endtime_sigmoid']>=end]
        num_associated_sigmoid=0
        #as long as the temporal search does not eliminate all possible related sigmoid events, proceed
        if sigmoid_search.empty==False:
                fl_point = Point((flare_set['hpc_x'].values[i], flare_set['hpc_y'].values[i]))
                min_s = spatial_sep
                event_index = None
                found_sigmoid = False
                for j in range(sigmoid_search.shape[0]):
                    #create a Shapely polygon object from sigmoid's location
                    sigmoid_poly = wkt.loads(sigmoid_search['hpc_bbox_sigmoid'].values[0])
                    #calculate the minimum 2D distance between the sigmoid's polygon and the flare's mean coordinates
                    chord = fl_point.distance(sigmoid_poly)
                    #calculate the minimum 3D distance along the sun's curved surface between the flare and sigmoid
                    #assumes the same radius for all events
                    s = r*np.arcsin(chord/(2*r))
                    #determine whether the spatial distance between flare and sigmoid meets the set parameter
                    if s <= spatial_sep:
                        #have found an associated sigmoid
                        found_sigmoid = True
                        num_associated_sigmoid+=1 
                        if s <= min_s: 
                            min_s = s
                            event_index = j
                if found_sigmoid:
                    flare_set['associated_sigmoid'].values[i] = sigmoid_search['SOL_standard'].values[event_index]
                    flare_set['num_associated_sigmoid'].values[i] = num_associated_sigmoid
                    flare_set['dist_sigmoid'].values[i] = min_s
                    for elem in sigmoid_keywords:
                        flare_set.loc[i, elem] = sigmoid_search[elem].values[event_index]
            
    #create boolean var to easily determine whether flare associated with an AR
    k = 0
    is_sigmoid = [0 for i in range(flare_set.shape[0])]
    for elem in flare_set['associated_sigmoid']:
        if elem!=None:
            is_sigmoid[k] = 1
        k+=1
    flare_set.loc[:, 'is_sigmoid'] = is_sigmoid
    
    #write dataframe to a csv file depending on initial parameters
    if output2file:
        if out_file == None:
            #create a generic name for file based on search parameters if no file name specified
             out_file = inputFile_fl[0:-4]+'_with_sigmoid.csv'
        #import which keywords to keep for outported data
        flare_keywords = list(np.genfromtxt('keywords_flare_with_goes_with_ar.csv', delimiter=',', dtype=str))
        #add to these keywords descriptors of associated sigmoid
        flare_keywords.extend(['is_sigmoid','associated_sigmoid', 'num_associated_sigmoid', 'dist_sigmoid'])
        flare_keywords.extend(sigmoid_keywords)
        #write to csv
        flare_set.to_csv(path_or_buf=out_file, columns = flare_keywords, index = False)
        
    return flare_set
    

In [21]:
# associate_sigmoid(inputFile_fl, inputFile_sigmoid, spatial_sep, output2file=False, out_file = None)
associate_sigmoid('flare_dataset_cleaned_30min_100arcsec_with_GOES_with_ar.csv', 'raw_sg.csv', 100, output2file=True,
                  out_file = None)

IOError: File flare_dataset_cleaned_30min_100arcsec_with_GOES_with_ar.csv does not exist